In [10]:
import numpy as np
import itertools as iter

def sqr_distance(loc1:dict, loc2:dict):
    r1 = np.array(list(loc1.values()))
    r2 = np.array(list(loc2.values()))
    dist2 = np.sum((r1 - r2)*(r1 - r2))
    return dist2

def do_contraction(T0:str, T1:str, direction:str):
    output = \
    f"            \n \
      {direction}     |     \n \
      |  --{T1}-- \n \
      |     |     \n \
      v  --{T0}-- \n \
            |        \
    "
    print(output)

def do_rgstep(loc1:dict, loc2:dict, direction:str):
    loc1_after, loc2_after = loc1.copy(), loc2.copy()
    loc1_after[direction] = loc1[direction] // 2
    loc2_after[direction] = loc2[direction] // 2
    sqr_distance_after = sqr_distance(loc1_after, loc2_after)
    sqr_distance_current = sqr_distance(loc1, loc2)
    if sqr_distance_after > 0:
        if loc1[direction] % 2 == 1:
            do_contraction("T0", "T1", direction)
        elif loc1[direction] % 2 == 0:
            do_contraction("T1", "T0", direction)

        if loc2[direction] % 2 == 1:
            do_contraction("T0", "T2", direction)
        elif loc2[direction] % 2 == 0:
            do_contraction("T2", "T0", direction)
    
    if sqr_distance_after == 0 and sqr_distance_current != 0:
        if loc1[direction] % 2 == 1:
            do_contraction("T2", "T1", direction)
        elif loc1[direction] % 2 == 0:
            do_contraction("T1", "T2", direction)
    elif sqr_distance_after == 0 and sqr_distance_current == 0:
        do_contraction("T1", "T0", direction)

nx = 2 #int(2**6) #- 1
ny = 5 #int(2**8) #- 1
nt = 6 #int(2**5) - 1
loc1 = {"T":nt, "X":nx, "Y":ny}

nx = int(2**6) #- 1
ny = int(2**8) #- 1
nt = int(2**5) #- 1
loc2 = {"T":nt, "X":nx, "Y":ny}

xloops = 10
yloops = 10
tloops = 10
TOT_RGSTEPS = {"T":tloops, "X":xloops, "Y":yloops}
rgstep = {"T":0, "X":0, "Y":0}

cycle = "TXY"
factor = {}
factor = {"(0,0,0)": 0}
for direction in iter.cycle("TXY"):
    if sum(rgstep.values()) >= sum(TOT_RGSTEPS.values()):
        break
    if rgstep[direction] < TOT_RGSTEPS[direction]:
        loc1_before, loc2_before = loc1.copy(), loc2.copy()
        do_rgstep(loc1, loc2, direction)
        rgstep[direction] += 1
        idx=f"({rgstep['T']},{rgstep['X']},{rgstep['Y']})"
        factor[idx] = sum(rgstep.values())
        loc1[direction] //= 2
        loc2[direction] //= 2
    print(f"impure tensor1: {loc1_before['X'], loc1_before['Y'], loc1_before['T']}->{loc1['X'], loc1['Y'], loc1['T']}")
    print(f"impure tensor2: {loc2_before['X'], loc2_before['Y'], loc2_before['T']}->{loc2['X'], loc2['Y'], loc2['T']}")
    print(f"square distance:", sqr_distance(loc1, loc2))


            
       T     |     
       |  --T0-- 
       |     |     
       v  --T1-- 
             |            
            
       T     |     
       |  --T0-- 
       |     |     
       v  --T2-- 
             |            
impure tensor1: (2, 5, 6)->(2, 5, 3)
impure tensor2: (64, 256, 32)->(64, 256, 16)
square distance: 67014
            
       X     |     
       |  --T0-- 
       |     |     
       v  --T1-- 
             |            
            
       X     |     
       |  --T0-- 
       |     |     
       v  --T2-- 
             |            
impure tensor1: (2, 5, 3)->(1, 5, 3)
impure tensor2: (64, 256, 16)->(32, 256, 16)
square distance: 64131
            
       Y     |     
       |  --T1-- 
       |     |     
       v  --T0-- 
             |            
            
       Y     |     
       |  --T0-- 
       |     |     
       v  --T2-- 
             |            
impure tensor1: (1, 5, 3)->(1, 2, 3)
impure tensor2: (32, 256, 16)->(32, 128, 16)
square dista

In [2]:
import opt_einsum as oe
class ATRG_base(object):
    def __init__(self, U, s, VH, ndim:int):
        self.U = U
        self.s = s
        self.VH = VH
        self.ndim = ndim
        assert ndim+1 == U.ndim, "legs do not match system dimension"


class impure_tensor(ATRG_base):
    def __init__(self, U, s, VH, ndim:int, loc:dict):
        super(impure_tensor, self).__init__(U, s, VH, ndim)
        self.loc = loc
        self.Ttype = "impureTensor"
        self.is_impureTensor = True

    def cal_location(self, direction:str):
        self.loc[direction] //= 2

    def trace(self):
        TrT = oe.contract("txyi,ij,jtxy", self.U, self.s, self.VH)
        return TrT


class pure_tensor(ATRG_base):
    def __init__(self, U, s, VH, ndim:int):
        super(pure_tensor, self).__init__(U, s, VH, ndim)
        self.Ttype = "pureTensor"
        self.is_impureTensor = False
    
    def trace(self):
        TrT = oe.contract("txyi,i,itxy", self.U, self.s, self.VH)
        return TrT

import numpy as np
dim = 3
U = np.random.rand(3,3,3,3)
s = np.random.rand(3)
VH = np.random.rand(3,3,3,3)

imp = impure_tensor(U, s, VH, 3, loc={"X":1, "Y":2, "T":5})


In [44]:
import numpy as xp
from functools import reduce
from operator import mul

def svd(A, shape):
    """
    Warning: Transpose the tensor A before svd it!
    split: If split == True, then return u*sqrt(s), s, sqrt(s)*vh, else return u, s, vh
    """
   
    llen = len(shape[0])
    rlen = len(shape[1])
    A_original_shape = A.shape

    A_newaxis   = tuple(shape[0]+shape[1])
    A_newshape  = [A.shape[i] for i in A_newaxis]
    A_mat_shape = (reduce(mul, A_newshape[:llen]), reduce(mul, A_newshape[llen:]))

    A_oldaxis = tuple([i for i in range(len(A.shape))])
    print(f"Transpose tensor A to new axis: {A_oldaxis}->{A_newaxis}")

    #transpose A to new axis
    A = xp.transpose(A, A_newaxis)

    #reshape tensor A to matrix then svd A
    A = xp.reshape(A, newshape=A_mat_shape)
    #u, s, vh = xp.linalg.svd(A, full_matrices=False)

    #reshape matrix A to tensor
    A = xp.reshape(A, newshape=A_newshape)

    #restore axis
    A_newaxis = xp.asarray(A_newaxis)
    restoreaxis = tuple(xp.argsort(A_newaxis))
    A = xp.transpose(A, restoreaxis)

    print(A_original_shape == A.shape)
    print("A restore", A.shape)

    return A


M = xp.random.rand(2,3,4,5,6,10)
M1 = svd(M, [[5,2,4,3,1],[0]])

xp.linalg.norm(M-M1)


Transpose tensor A to new axis: (0, 1, 2, 3, 4, 5)->(5, 2, 4, 3, 1, 0)
True
A restore (2, 3, 4, 5, 6, 10)


0.0

In [10]:
import numpy as np
from scipy.special import roots_legendre


def integral(func, x, w):
    fi = w * func(x)
    I = np.sum(fi)
    return I

def func1(x):
    return np.sqrt(1 - x**2)

def func2(x):
    return 1 / np.sqrt(1 - x**2)

K = 49
N = (K - 1) // 2
h = 1.59 / N

K = 2 * N + 1
tDE = np.linspace(-N * h, N * h, num = K, endpoint = True)
xDE = np.tanh( (np.pi / 2) * np.sinh(tDE) )
wDE = h * (np.pi / 2) * np.cosh(tDE) / (np.cosh((np.pi / 2) * np.sinh(tDE))**2)

err1DE = abs((integral(func1, xDE, wDE) - np.pi / 2) / np.pi / 2)
err2DE = abs((integral(func2, xDE, wDE) - np.pi) / np.pi)

tGL, wGL = roots_legendre(K)
xGL = tGL
err1GL = abs((integral(func1, xGL, wGL) - np.pi / 2) / np.pi / 2)
err2GL = abs((integral(func2, xGL, wGL) - np.pi) / np.pi)

print(f"Double exponential formula : err1={err1DE:.6e}, err2={err2DE:.6e}")
print(f"Gauss Legendre quadrature  : err1={err1GL:.6e}, err2={err2GL:.6e}")


Double exponential formula : err1=8.552591e-06, err2=2.767608e-02
Gauss Legendre quadrature  : err1=1.081214e-06, err2=1.119776e-02


In [22]:
eps = 1e-12
for N in range(2, 30):
    K = 2 * N + 1
    print(f"N={N}:")
    for i in range(10):
        h = 1 / (2**i)
        tDE = np.linspace(-N * h, N * h, num = K, endpoint = True)
        xDE = np.tanh( (np.pi / 2) * np.sinh(tDE) )
        wDE = h * (np.pi / 2) * np.cosh(tDE) / (np.cosh((np.pi / 2) * np.sinh(tDE))**2)

        err1DE = abs((integral(func1, xDE, wDE) - np.pi / 2) / np.pi / 2)
        print(f"h = 1 / 2^{i}, err1={err1DE:.6e}")
    print()

N=2:
h = 1 / 2^0, err1=2.255600e-02
h = 1 / 2^1, err1=1.878640e-04
h = 1 / 2^2, err1=2.831969e-02
h = 1 / 2^3, err1=1.080897e-01
h = 1 / 2^4, err1=1.737863e-01
h = 1 / 2^5, err1=2.111804e-01
h = 1 / 2^6, err1=2.304992e-01
h = 1 / 2^7, err1=2.402382e-01
h = 1 / 2^8, err1=2.451177e-01
h = 1 / 2^9, err1=2.475587e-01

N=3:
h = 1 / 2^0, err1=2.255600e-02
h = 1 / 2^1, err1=1.782693e-05
h = 1 / 2^2, err1=6.723049e-03
h = 1 / 2^3, err1=6.809609e-02
h = 1 / 2^4, err1=1.458556e-01
h = 1 / 2^5, err1=1.959886e-01
h = 1 / 2^6, err1=2.227415e-01
h = 1 / 2^7, err1=2.363388e-01
h = 1 / 2^8, err1=2.431654e-01
h = 1 / 2^9, err1=2.465822e-01

N=4:
h = 1 / 2^0, err1=2.255600e-02
h = 1 / 2^1, err1=1.811128e-05
h = 1 / 2^2, err1=1.084192e-03
h = 1 / 2^3, err1=3.971249e-02
h = 1 / 2^4, err1=1.202523e-01
h = 1 / 2^5, err1=1.811252e-01
h = 1 / 2^6, err1=2.150260e-01
h = 1 / 2^7, err1=2.324447e-01
h = 1 / 2^8, err1=2.412138e-01
h = 1 / 2^9, err1=2.456058e-01

N=5:
h = 1 / 2^0, err1=2.255600e-02
h = 1 / 2^1, err

/tmp/ipykernel_3533982/3286956882.py:9: RuntimeWarning: overflow encountered in cosh
  wDE = h * (np.pi / 2) * np.cosh(tDE) / (np.cosh((np.pi / 2) * np.sinh(tDE))**2)
/tmp/ipykernel_3533982/3286956882.py:9: RuntimeWarning: overflow encountered in square
  wDE = h * (np.pi / 2) * np.cosh(tDE) / (np.cosh((np.pi / 2) * np.sinh(tDE))**2)


In [31]:
eps = 1e-12
for N in range(2, 30):
    K = 2 * N + 1
    print(f"N={N}:")
    for i in range(10):
        h = 1 / (2**i)
        tDE = np.linspace(-N * h, N * h, num = K, endpoint = True)
        xDE = np.tanh( (np.pi / 2) * np.sinh(tDE) )
        wDE = h * (np.pi / 2) * np.cosh(tDE) / (np.cosh((np.pi / 2) * np.sinh(tDE))**2)

        if i == 0:
            I0  = integral(func1, xDE, wDE)
            J0  = h * sum(func1(xDE) * 2 / (np.cosh(np.sinh(tDE) * np.pi / 2)**2))
            err = 1.0
        else:
            I1  = integral(func1, xDE, wDE)
            J1  = h * sum(func1(xDE) * 2 / (np.cosh(np.sinh(tDE) * np.pi / 2)**2))
            #err = abs((I1 - I0) / I1)
            err = max(abs(I1 - I0), abs(J1 - J0)) / abs(I1)
            I0 = I1
            J0 = J1

        err_abs = abs((integral(func1, xDE, wDE) - np.pi / 2) / np.pi / 2)
        print(f"h = 1 / 2^{i}, relative error={err:.6e}, absolute error={err_abs:.6e}")
    print()

N=2:
h = 1 / 2^0, relative error=1.000000e+00, absolute error=2.255600e-02
h = 1 / 2^1, relative error=1.611821e-01, absolute error=1.878640e-04
h = 1 / 2^2, relative error=1.269027e-01, absolute error=2.831969e-02
h = 1 / 2^3, relative error=6.482098e-01, absolute error=1.080897e-01
h = 1 / 2^4, relative error=1.068637e+00, absolute error=1.737863e-01
h = 1 / 2^5, relative error=1.218176e+00, absolute error=2.111804e-01
h = 1 / 2^6, relative error=1.259212e+00, absolute error=2.304992e-01
h = 1 / 2^7, relative error=1.269716e+00, absolute error=2.402382e-01
h = 1 / 2^8, relative error=1.272358e+00, absolute error=2.451177e-01
h = 1 / 2^9, relative error=1.273019e+00, absolute error=2.475587e-01

N=3:
h = 1 / 2^0, relative error=1.000000e+00, absolute error=2.255600e-02
h = 1 / 2^1, relative error=1.606042e-01, absolute error=1.782693e-05
h = 1 / 2^2, relative error=2.770865e-02, absolute error=6.723049e-03
h = 1 / 2^3, relative error=3.611090e-01, absolute error=6.809609e-02
h = 1 / 2

/tmp/ipykernel_3533982/3219605622.py:9: RuntimeWarning: overflow encountered in cosh
  wDE = h * (np.pi / 2) * np.cosh(tDE) / (np.cosh((np.pi / 2) * np.sinh(tDE))**2)
/tmp/ipykernel_3533982/3219605622.py:13: RuntimeWarning: overflow encountered in cosh
  J0  = h * sum(func1(xDE) * 2 / (np.cosh(np.sinh(tDE) * np.pi / 2)**2))
/tmp/ipykernel_3533982/3219605622.py:9: RuntimeWarning: overflow encountered in square
  wDE = h * (np.pi / 2) * np.cosh(tDE) / (np.cosh((np.pi / 2) * np.sinh(tDE))**2)
/tmp/ipykernel_3533982/3219605622.py:17: RuntimeWarning: overflow encountered in square
  J1  = h * sum(func1(xDE) * 2 / (np.cosh(np.sinh(tDE) * np.pi / 2)**2))
/tmp/ipykernel_3533982/3219605622.py:17: RuntimeWarning: overflow encountered in cosh
  J1  = h * sum(func1(xDE) * 2 / (np.cosh(np.sinh(tDE) * np.pi / 2)**2))
